In [1]:
import torch
import numpy as np
import copy
import matplotlib.pyplot as plt

from torch.utils.data import Dataset


from utilities import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


/home/qguo0701/.conda/envs/qingguo/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
batch_size = 128

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

In [4]:
args = dict()

args['dim_z'] = 5

In [5]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [6]:
from sklearn.cross_decomposition import CCA

In [7]:
cca = CCA(n_components=args['dim_z'])

In [8]:
X_train = train_data.data
X_train_left = X_train[:,:,:14]
X_train_right = X_train[:,:,14:]
Y_train = train_data.targets
X_test = test_data.data
X_test_left = X_test[:,:,:14]
X_test_right = X_test[:,:,14:]
Y_test = test_data.targets

In [9]:
X_train_left = X_train_left.numpy().reshape(len(X_train),-1)
X_train_right = X_train_right.numpy().reshape(len(X_train),-1)
X_test_left = X_test_left.numpy().reshape(len(X_test),-1)
X_test_right = X_test_right.numpy().reshape(len(X_test),-1)

In [10]:
X_train_data = list(zip(torch.Tensor(X_train_left).to(device),torch.Tensor(X_train_right).to(device)))
X_test_data = list(zip(torch.Tensor(X_test_left).to(device),torch.Tensor(X_test_right).to(device)))

In [11]:
X_train_loader = torch.utils.data.DataLoader(X_train_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True)
X_test_loader = torch.utils.data.DataLoader(X_test_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True)

In [12]:
# X_train_left.shape
cca.fit(X_train_left, X_train_right)

/home/qguo0701/.conda/envs/qingguo/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:77: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


CCA(n_components=5)

In [33]:
# cca.fit
Z_train_left,Z_train_right = cca.transform(X_train_left,X_train_right)
Z_test_left,Z_test_right = cca.transform(X_test_left,X_test_right)

In [34]:
Z_train = np.concatenate([Z_train_left,Z_train_right],1)
Z_test = np.concatenate([Z_test_left,Z_test_right],1)

In [35]:
data_dict = dict()
data_dict['Z_train'] = Z_train
data_dict['Y_train'] = Y_train
data_dict['Z_test'] = Z_test
data_dict['Y_test'] = Y_test
data_dict['method'] = 'CCA'

In [37]:
np.save('feature/cca_d_5.npy',data_dict)